# TopoGravCorrelator
Welcome! This is a project that is supposed to explore the correlation between land elevation (topographic elevation) and gravity anomalies. I'll be adding more to this as time goes on. Hopefully I finish it!

**Grace Dataset**

D. N. Wiese, D.-N. Yuan, C. Boening, F. W. Landerer, M. M. Watkins. 2023. JPL GRACE and GRACE-FO Mascon Ocean, Ice, and Hydrology Equivalent Water Height CRI Filtered RL06.3Mv04. Ver. RL06.3Mv04. PO.DAAC, CA, USA. Dataset accessed [2025-05-28] at https://doi.org/10.5067/TEMSC-3JC634

**Teopography dataset**

NOAA National Centers for Environmental Information. 2022: ETOPO 2022 15 Arc-Second Global Relief Model. NOAA National Centers for Environmental Information. DOI: 10.25921/fd45-gt74. Accessed [6/1/2025].

In [ ]:
print('Hello World!')

# The sweet sweet math/physics
I'll start this off by going over the background math and physics. I learned all of this from Geophysics class, Geodynamics (2nd edition) by Donald L. Turcotte and Geral Schubert, and The Solid Earth (2nd edition) by C.M.R. Fowler.

### Gravity 

- **Gravity of the Earth**  
    The gravitational acceleration towards the Earth can be represented by:

    $$
    -a = \frac{GM_e}{r^2}
    $$

    Where:
    - $G$ is the gravitational constant.
    - $M_e$ is the mass of the Earth.
    - $r$ is the radius of the Earth.

    This is a nice representation if Earth were a perfect shape and stationaryy. But alas, it is neither of those things. First of all, the Earth is not a perfect sphere. It's more like an oblate spheroid. We can approximate this shape by taking an ellipse and revolving it around its minor axis...like so:

    $$
    f = \frac{R_e - R_p}{R_e}
    $$
    
    This is called ellipticity, or polar flattening, given by $f$, where:
    - $R_e$ is the equatorial radius.
    - $R_p$ is the polar radius.

    We can also describe the radius of an oblate spheroid as:

    $$
    r = R_e (1 - fsin^2 \lambda)
    $$

    Something interesting that happens with the Earth...is that it spins. I know, I know, it's not that impressive, but this does something neat with the gravity. Because the Earth is spinning, gravity on its surface is actually reduced due to Centrifugal Acceleration. On a sphere with an angular rotation frequency $\omega$, this turns out to be:

    $$
    g_{rot} = g - \omega^2 R_e cos^2 \lambda
    $$

    where:
    - $g$ is the gravity on a non-rotating sphere.
    - $\lambda$ is the lattitude.

    Remember that this is for a nice sphere. It's a bit trickier to approximate for an oblate spheroid. Luckily the International Association of Geodesy adopted the reference gravity formula, which says:

    $$
    g(\lambda) = g_e (1 + \alpha sin^2 \lambda + \beta sin^4 \lambda)
    $$

    Where:
    - $g_e$ is the equatorial gravitational acceleration (which is about 9.7803185 m/s^2)
    - $alpha$ is a constant accounting for the first-order effects on Earth's oblateness (like how gravity is stronger at the poles than at the equator). The numerical value is $\alpha = 5.278895 * 10^{-3}$
    - $\beta$ is a constant that accounts for higher-order oblateness, more along the lines of how rotation and ellipticity mess with gravity. The numerical value is $\beta = 2.3462 * 10^{-5}$

    This pretty much sets gravity as a function of lattitude, which depends on rotation, ellipticity, distance from the equator, and many other things. I shall do a derivation of this in its own subsection.



### Satellite Orbits
   Since this project relies on satellite data, it'll be worthwhile to have atleast a basic grasp of the orbits of satellites around the Earth. I'll have it simplified for now...since I don't really have a great understanding of it myself. So, ltet's consider the gravity of the Earth acting on a tiny little satellite. Re-arranging our little equation for Earth's gravity from earlier, we get:

   $$
      \frac{G M_e m}{r^2} = m \omega^2 r
   $$

   Where Earth's gravity acting on the satellite is being balanced by the centrifugal force the satellite experiences while orbiting:
    - $m$ is the mass of the satellite.
    - $\omega$ is the angular velocity of the satellite.

   We can re-arrange this equation to solve for $\omega$:

   $$
    \omega = (\frac{G M_e}{r^3})^{1/2}
   $$

   This shows the angular velocity that a satellite needs to orbit comfortably around a perfectly spherical Earth. With this project, I'm using Two-Line Elements(TLEs) to simulate satellite orbits and groundtracks. Let's look at the TLE for the GRACE-FO 1 that I'm using.
   ```
      GRACE-FO 1              
      1 43476U 18047A   25184.60280784  .00002951  00000+0  86417-4 0  9995
      2 43476  88.9777 246.0966 0013351  41.2483 318.9776 15.34771716396270
   ```
   I know what you must be thinking. How does one read this? That's exactly what I was, and still am, thinking.

In [1]:
from IPython.display import display, HTML
display(HTML(""" 
<div style="display: flex;">
    <div>
        <img src="Assets/TLE-Breakdown-Mahdi,2015.png" width="1000">
    </div>
</div>         
"""))

### Image reference. Go check this paper out, it's great:
# @article{article,
# author = {Chessab Mahdi, Mohammed},
# year = {2015},
# month = {01},
# pages = {1-8},
# title = {TIGRISAT Orbital MotionSimulation and Analysis},
# volume = {5},
# journal = {Journal of Control Engineering and Technology},
# doi = {10.14511/jcet.issue.050101}
# }


This happy little image (courtesy of Chessab Mahdi, Mohammed. (2015). TIGRISAT Orbital MotionSimulation and Analysis. Journal of Control Engineering and Technology. 5. 1-8. 10.14511/jcet.issue.050101. Go check it out, it's a great paper), is a visual guide that tells you what numbers in a TLE tell you (or what they should be telling you). These three lines give you all of the orbital information needed to simulate a satellite's position over time! In theory anyway, but how do you really do it? Well, luckily for us, we have numerous tools that can help us with this. In my case, I used the Skyfield python package to simulate the GRACE-FO1 and ICEsat-2 orbits, which you can see simulated in the little animation and groundtrack figures below:

In [2]:
from IPython.display import display, HTML

display(HTML("""
<div style="display: flex; gap: 20px; align-items: center;">
     <div>
          <img src="Assets/Orbital_dual.gif" width="800">
     </div>
     <div>
          <img src="Assets/Satellite Groundtracks.png" width="1200">
     </div>
</div>  
"""))

**Satellite Ground Tracks**
Now hold on now, we can't go any further without understanding what's actually going on here! So...how do we actually come about these orbits just from three lines of information? Well, first consider this. From the TLE we have:

| Parameter Name | Variable |
| ----------------- | ----------------- |
| Inclination | $i$ |
| RAAN (Right Ascension of Ascending Node) | $\Omega$ |
| Eccentricity | $e$ |
| Argument of Perigee | $\omega$ |
| Mean Anomaly | $M$ |
| Mean Motion (revolutions / day) | $n$ |

Looks nice, but what can we do with this? Well, let's take a look at this image from Astrodynamics by Roger R. Bate, Donald D. Mueller, and Jerry E. White to get an idea of what we can do:

In [3]:
from IPython.display import display, HTML

display(HTML("""
<div style="display: flex; gap: 20px; align-items: center;"">
     <div>
          <img src="Assets/Satellite-Guidance-Astrodynamics.png" width="800">
     </div>
     <div>
          <img src="Assets/Anomalies.gif" width="1200">
     </div>
</div>  
"""))

Though this does use slightly different terminology, in this specific case, they are equivalent to what we're using with the TLE. As an example, the book uses Argument of Periapses in place of our Argument of Perigee, and Longitude of the Ascending Node (LAN) in place of our (RAAN). Periapsis is a more general term for the point in orbit where the orbiter is closest to the celestial object. Perigee is a term specifically refering to an object at the closest point in its orbit around Earth. So if someone used perigee when describing an orbit, you'd know that they're referring to an object orbiting Earth. In the  left figure this is showed as $\omega$ between the ascending node and periapsis direction, and is measured along the satellite's direction.

The difference between LAN and RAAN, however, is a bit more complicated. LAN refers to the angle between a reference direction, whcih is the origin longitude, and the ascending node. RAAN is similar, but it uses a specific reference direction, being the vernal equinox as the origin of longitude. They aren't quite the same thing, but if the LAN uses the vernal equinox as the origin of longitude, then it can be equivalent to the RAAN. In the left figure it is shown as the angle between the vernal equinox and the ascending node. Essentially this is the point where the satellite rises up through the equatorial plane. 

The vernal equinox direction...is a bit of a bridge of different disciplines describing the same thing in different ways. In the case of a TLE, which would be a geometric definition, is the vector from the center of the Earth to the center of the Sun on the first light of Spring. In the image, it is depicted as the angle $\Omega$ between the vernal equinox vector and the ascending node.

From here we can also see the eccentricity $e$, which is interestingly represented as the vector $\bar{e}$ from the center of the Earth to periapsis. Eccentricity, in the context of orbits, is the measure of how stretched the orbit is. If you have $ e = 0$, then you'll have a perfectly circular orbit (the orbit of your fantasies). The closer the parameter gets to $e = 1$, he more stretched  the orbit becomes, with an $e = 1$ resulting in a parabolic trajectory. The parabolic trajectory means the satellite the exact energy needed to escape the (in this case) Earth's gravitational influence, but has zero remaining mechanical energy. This means that after escape, it remains on that trajectory to infinity while continuously slowing down, but it never reaches zero. If the parameter is $e > 1$ the satellite is on a hyperbolic escape trajectory with energy to spare, keeping it at a minimum velocity. When represented as a vector, essentricity is dependent on: 
    $$
    \vec{e} = \frac{1}{\mu} (\vec{v} \times \vec{h}) - \frac{\vec{r}}{|\vec{r}|}
    $$
Where:
| Parameter | Representation |
| ----------------- | ----------------- |
| Specific Angular Momentum | $\vec{h} = \vec{r} \times \vec{v}$ |
| Unit vector from central mass to satellite | $ \frac{\vec{r}}{\|\vec{r}\|} $ |
| Velocity vector | $\vec{v}$ |
| Position vector | $\vec{r}$ |
| Gravitational parameter of central mass | $\mu = G * M$ |

So what does $\vec{v} \times \vec{h}$ result in? Well, it will result in a vector that's perpendicular to the angular momentum vector $\vec{h}$. This next one is a bit tricky and it requires us to think about units. $\frac{1}{\mu}$ is a conversion factor to handle units and the magnitude of the vector. If you have a velocity vector $\vec{v}$, it's probably going to have some units like $\frac{km}{sec}$ tied to it. Your specific angular momentum vector $\vec{h}$ will probably also have units tied to it, which could be $\frac{km^2}{sec}$. When you cross these two vectors, you still multiply the units, which would result in $\frac{km^3}{sec^2}$. Eccentricity $e$ is, itself, a scalar that represents both the shape of the orbit and the magnitude of its vector counterpart $\vec{e}$. Both of these are dimensionless in nature because they describe direction and shape. To make $\vec{e}$ unitless, you need to scale it by dividing by the gravitational parameter of your central mass (which should be in the same units). In this gase, it should be $\frac{km^3}{sec^2}$, resulting in a dimensionless parameter. After this, you subtract the unit vector $\frac{\vec{r}}{\|\vec{r}\|}$, which is the unit position vector pointing towards periapsis (as the $\vec{e}$ does in the figure above) to keep it pointing towards the periapsis of your orbit. Then bamn, you have your eccentricity vector that points towards periapsis! The TLE does this work for you though by giving you the scalar $e$.

Alright, with all that...what do we do from here to arrive at our obit for our satellite? Well, the first thing we can do is solve for our semi-major axis. The semi-major axis of an orbit depends a bit on the type of orbit you're working with. If you're looking at a circular orbit, it will just be the orbital radius. However, if your orbit is elliptical, the semi-major axis will be half the length of the major axis (which is the longest diameter). There are quite a few methods you can use to calculate it depending on what you know about your orbit. Since we know both the gravitational parameter $/mu$ and the mean motion $n$, we can calculate the length of the semi-major axis by:
    $$
        a = (\frac{\mu}{\frac{2 \pi n}{86400}})^{1/3}
    $$
You want the mean motion converted into radians per second, due to Kepler's third law assuming angular motion in radians and gravity parameter $u$ using seconds. Mean motion $n$ goes by revolutions per day, so you want to convert from revolutions to radians, and days to seconds. This calculation will result in a semi-major axis in kilometers. From here, it gets a bit tricky. Now that we have a bit of information about the shape of this orbit...well, wouldn't you like to know about the position of the satellite in the orbit? That is, after all, the purpose of a TLE, right? Well, good news! We can get that in only three...not-so-simple steps! We'll want to know about the True, Mean, and Eccentric Anomalies of the orbit. I'm gonna split these up a bit, because I want to try and do them justice.

- **Mean Anomaly**
    The Mean Anomaly is a measure of time. Specifically, the time that has passed since our satellite passed its periapsis. It can also tell us were our satellite would be at a specific time after passing periapsis, given a constant angular speed of course. Dpending on how you approach finding an orbit, you may have to solve for this, which there are several methods to do so. For example, if you know the mean motion $n$, time since the satellite passed periapsis (can call this $s$), and the current time $t$, you could solve for Mean Anomaly by:
    $$
        M = n(t - s)
    $$
    But by a happy coincidence, we have already been given the Mean Anomaly by the TLE. It is important to note that this only gives a position in time along the orbit, not the position in space.

- **Eccentric Anomaly**
    This is the tricky one. The Eccentric Anomaly is an angular parameter that describes the position of an object moving along a Keplerian orbit. But here's a question for you: What is the the reference frame of the eccentricity anomaly? It's not the focus, instead it's the center of the ellipse. So how do we get the eccentric anomaly E? Well, we first gotta understand Kepler's equation.
        $$
            M = E - esin(E)
        $$
    We need to solve this for E. But why do we need to solve this one for E?
    

In [4]:
from IPython.display import display, HTML

display(HTML("""
<div style="display: flex; gap: 20px; align-items: center;"">
     <div>
          <img src="Assets/Kepler's Equation 1.png" width="400">
     </div>
     <div>
          <img src="Assets/Kepler's Equation 3.png" width="430">
     </div>
    <div>
          <img src="Assets/Kepler's Equation 2.png" width="410">
     </div>
</div>  
"""))

- **Kepler's Equation**
Looking at the three above figures (from the video "Kepler's Impossible Equation" by Welch Labs on Youtube), we can start to understand the geometry of this.

| Parameter | Representation |
| ----------------- | ----------------- |
| Elliptical Orbit Area| $A1$ |
| Leftover Area | $A2$ |
| Circular Orbit Area| $A3$ |
| True Anomaly | $\theta$ |

Looking at the three figures above, we can say that the area of the elliptical orbit ($A1$) plus the leftovcer area ($A2$) is proportional to the area of a fully circular orbit ($A3$).
    $$
        \frac{A1 + A2}{A3} = \frac{b}{a}
    $$


In [5]:
from IPython.display import display, HTML

display(HTML("""
<div style="text-align: center;">
    <img src="Assets/grace_lwe_animation.gif" width="1000"><br>
    <p style="font-weight: bold;">LWE Thickness</p>
    <br>
    <img src="Assets/LWE Gravity Change.gif" width="1000"><br>
    <p style="font-weight: bold;">Gravity Change</p>
</div>
"""))

In [6]:
from IPython.display import display, HTML

display(HTML("""
<div style="display: flex;">
    <div>
        <img src="Assets/ETOPO_downsample.png" width="1000">
    </div>
</div>
"""))